In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#  **Cleaning Dataset** #

In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, FunctionTransformer, MultiLabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import ast

# Define preprocessing functions and classes
def clean_percentage(x):
    if isinstance(x, str):
        return float(x.strip('%')) / 10
    return x

def clean_comma(x):
    if isinstance(x, str):
        return float(x.replace(',', ''))
    return x

class MultiLabelBinarizerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        self.mlb.fit(X)
        return self

    def transform(self, X):
        return self.mlb.transform(X)

# Load the data
df = pd.read_csv('/kaggle/input/dataset-7-july/updated_final_combined_dataset_RT.csv')
df.drop(columns=['movie','poster_url'], inplace=True)

print(len(df))

df.dropna(subset=['more_like_this', 'release_date', 'star', 'director','gross(in $)'], inplace=True)
print(len(df))


df.dropna(subset=['audience_score', 'critics_score'], how='all')
print(len(df))

df = df.drop(df[df['original_language'] != 'English'].index)
df.drop(columns=['original_language'], inplace=True)
print(len(df))

def str_to_list(s):
    s = s.strip()  # Remove leading/trailing whitespace
    if s.startswith('[') and s.endswith(']'):
        try:
            return ast.literal_eval(s)
        except (ValueError, SyntaxError):
            return s  # Return the original string if evaluation fails
    return s  # Return the original string if it's not a list format

# Apply the function to the column
df['genre'] = df['genre'].apply(str_to_list)
df['star'] = df['star'].apply(str_to_list)
df['more_like_this'] = df['more_like_this'].apply(str_to_list)
# Fill missing values and clean data
df['critics_score'] = df['critics_score'].apply(lambda x: np.nan if isinstance(x, str) and 'not found' in x else x)
df['audience_score'] = df['audience_score'].apply(lambda x: np.nan if isinstance(x, str) and 'not found' in x else x)
df['audience_score'] = df['audience_score'].apply(clean_percentage)
df['critics_score'] = df['critics_score'].apply(clean_percentage)
df['audience_score'] = df['audience_score'].fillna(df['audience_score'].mean())
df['critics_score'] = df['critics_score'].fillna(df['critics_score'].mean())
df['description'] = df['description'].fillna('')
df['popularity_score'] = df['popularity_score'].apply(clean_comma)
df['gross(in $)'] = df['gross(in $)'].apply(clean_comma)
df['star'] = df['star'].apply(lambda x: x.split(', \n') if isinstance(x, str) else x if isinstance(x, list) else [])
df['more_like_this'] = df['more_like_this'].apply(lambda x: x.split(', \n') if isinstance(x, str) else x if isinstance(x, list) else [])

df['production_companies'] = df['production_companies'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])
df['genre'] = df['genre'].apply(lambda x: x if isinstance(x, list) else x.split(', ') if isinstance(x, str) else [])
df['release_year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year
df['release_year'] = df['release_year'].fillna(df['release_year'].median())
df = df.reset_index(drop=True)


20419
10498
10498
8246


20419
9631
9631
7610

In [26]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, FunctionTransformer, MultiLabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import ast

def str_to_list(s):
    s = s.strip()  # Remove leading/trailing whitespace
    if s.startswith('[') and s.endswith(']'):
        try:
            return ast.literal_eval(s)
        except (ValueError, SyntaxError):
            return s  # Return the original string if evaluation fails
    return s  
df = pd.read_csv('/kaggle/input/dataset-7-july/updated_final_combined_dataset_RT.csv')
df = df[df['popularity_score'].notna()]
df['more_like_this'] = df['more_like_this'].apply(str_to_list)


In [13]:
df

,movie_name,movie_id,runtime,genre,rating,director,star,gross(in $),movie,description,audience_score,critics_score,original_language,where_to_watch,poster_url,popularity_score,release_date,production_companies,budget($),more_like_this
13,Over the Hedge,tt0327084,83 min,"Animation, Adventure, Comedy",6.7,"Tim Johnson, \nKarey Kirkpatrick","Bruce Willis, \nGarry Shandling, \nSteve Carel...",155019340.0,Over the Hedge,When Verne (Garry Shandling) and fellow woodla...,71%,75%,English,"[{'platform': 'amazon-prime-video-us', 'href':...",https://m.media-amazon.com/images/M/MV5BMWVhNT...,"2,842","May 19, 2006","['DreamWorks Animation', 'Pacific Data Images ...",80000000.0,"[Flushed Away, Open Season, Bee Movie, Monster..."
19,Casino Royale,tt0381061,144 min,"Action, Adventure, Thriller",8.0,Martin Campbell,"Daniel Craig, \nEva Green, \nJudi Dench, \nJef...",167445960.0,Casino Royale,"After receiving a license to kill, British Sec...",90%,94%,English,"[{'platform': 'amazon-prime-video-us', 'href':...",https://m.media-amazon.com/images/M/MV5BZmI0MT...,704,"November 17, 2006","['Columbia Pictures', 'Eon Productions', 'Casi...",150000000.0,"[Skyfall, Quantum of Solace, Spectre, No Time ..."
24,Happy Feet,tt0366548,108 min,"Animation, Adventure, Comedy",6.4,"George Miller, \nWarren Coleman, \nJudy Morris","Elijah Wood, \nBrittany Murphy, \nHugh Jackman...",198000317.0,Happy Feet,"Mumble (Elijah Wood), a young emperor penguin,...",70%,76%,English,"[{'platform': 'amazon-prime-video-us', 'href':...",https://m.media-amazon.com/images/M/MV5BZWU2ND...,"2,047","November 17, 2006","['Warner Bros.', 'Village Roadshow Pictures', ...",100000000.0,"[Happy Feet Two, Surf's Up, Bee Movie, Shark T..."
26,DOA: Dead or Alive,tt0398913,87 min,"Action, Adventure, Mystery",4.8,Corey Yuen,"Jaime Pressly, \nDevon Aoki, \nSarah Carter, \...",480314.0,DOA: Dead or Alive,Four beautiful rivals at an invitation-only ma...,36%,33%,English,"[{'platform': 'amazon-prime-video-us', 'href':...",https://m.media-amazon.com/images/M/MV5BMTIyND...,"3,719","June 15, 2007","['Dimension Films', 'Constantin Film', 'Impact...",21000000.0,"[Jack Stall Dead, The Law of Destiny, Out for ..."
29,Running Scared,tt0404390,122 min,"Action, Crime, Drama",7.3,Wayne Kramer,"Paul Walker, \nCameron Bright, \nChazz Palmint...",6855137.0,Running Scared,Troops ready to invade Cuba search the Evergla...,50%,NaN,English,[],https://m.media-amazon.com/images/M/MV5BMTIwOT...,"4,589","February 24, 2006","['New Line Cinema', 'Media 8 Entertainment', '...",15000000.0,"[Alpha Dog, Joy Ride, Into the Blue, Smokin' A..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19962,The Wiz,tt0078504,134.0,"['Adventure', 'Family', 'Fantasy']",5.5,Sidney Lumet,"['Diana Ross', 'Michael Jackson', 'Nipsey Russ...",21049053.0,The Wiz,When Harlem schoolteacher Dorothy (Diana Ross)...,72%,91%,NaN,[],https://m.media-amazon.com/images/M/MV5BNTRmNm...,"4,917","October 24, 1978","['Universal Pictures', 'Motown Productions']",24000000.0,"[The Wiz Live!, Jackson 5ive, The Wizard of Oz..."
20061,The Ice Storm,tt0119349,112.0,['Drama'],7.3,Ang Lee,"['Kevin Kline', 'Joan Allen', 'Sigourney Weave...",8038061.0,The Ice Storm,"In the 1970s, an outwardly wholesome family be...",82%,86%,English,"[{'platform': 'amazon-prime-video-us', 'href':...",https://m.media-amazon.com/images/M/MV5BYTU1Yz...,"4,546","November 26, 1997","['Searchlight Pictures', 'Good Machine', 'Cana...",18000000.0,"[The Wedding Banquet, Blood Simple, In the Bed..."
20242,Sexy Sisters,tt0075322,85.0,"['Crime', 'Drama']",4.4,Jesús Franco,"['Pamela Stanford', 'Karine Gambier', 'Jack Ta...",NaN,Sexy Sisters,"In the late '60s, the self-proclaimed belles o...",42%,48%,English,"[{'platform': 'amazon-prime-video-us', 'href':...",https://m.media-amazon.com/images/M/MV5BMTc2OT...,"4,272","September 2, 1977",['Elite Film'],NaN,"[Nine Guests for a Crime, Jack the Ripper, The..."
20336,The Life Aquatic with Steve Zissou,tt0362270,119.0,"['Acti

In [4]:
def find_columns_with_lists(df):
    columns_with_lists = []
    for column in df.columns:
        if any(isinstance(i, list) for i in df[column]):
            columns_with_lists.append(column)
    return columns_with_lists

columns_with_lists = find_columns_with_lists(df)
print("Columns with lists:", columns_with_lists)

Columns with lists: ['genre', 'star', 'production_companies', 'more_like_this']


In [3]:
df.isnull().sum()


movie_name                0
movie_id                  0
runtime                  27
genre                     0
rating                   33
director                  3
star                      0
gross(in $)             314
movie                   222
description               5
audience_score           44
critics_score           180
original_language        22
where_to_watch            0
poster_url                0
popularity_score          0
release_date             10
production_companies      0
budget($)               512
more_like_this            0
dtype: int64

#  **Preprocessing features**

In [27]:
import pandas as pd

# Example DataFrame setup (assuming df is already defined)
# df = pd.DataFrame({
#     'movie_name': ['Movie1', 'Movie2', 'Movie3'],
#     'more_like_this': [['Movie2', 'Movie4'], ['Movie1', 'Movie3'], ['Movie5', 'Movie6']]
# })
df = df[df['genre'].apply(lambda x: len(x) > 0)]

valid_movies_set = set(df['movie_name'])

def filter_movies(movie_list, valid_movies):
    valid_movie_list = [movie for movie in movie_list if movie in valid_movies]
    invalid_movie_list = [movie for movie in movie_list if movie not in valid_movies]
    return valid_movie_list, invalid_movie_list

# Apply the function to the more_like_this column and unpack the results
df[['more_like_this', 'invalid_more_like_this']] = df['more_like_this'].apply(lambda row: pd.Series(filter_movies(row, valid_movies_set)))

# Filter the DataFrame to keep rows with valid movies
df = df[df['more_like_this'].apply(lambda x: len(x) > 0)]

# Create a list of all invalid movies
invalid_movies = df['invalid_more_like_this'].explode().dropna().unique().tolist()

# Display the resulting DataFrame and the list of invalid movies



In [22]:
invalid_movies

['Sleepwalkers',
 'Bitter Moon',
 'Destroy All Monsters',
 'The Best Exotic Marigold Hotel',
 "God's Not Dead",
 'City of Lies',
 'Limitless',
 'Coup de Chance',
 'Left Behind',
 'Ernest Goes to Jail',
 'Colors of Evil: Red',
 'The Bubble',
 'Sick of Myself',
 'The Beach Boys',
 'Halloween Ends',
 'Chief of Station',
 'Do Not Expect Too Much from the End of the World']

In [29]:
invalid_movies  = pd.DataFrame(invalid_movies , columns=['movie_name'])

# Save the DataFrame as a CSV file
invalid_movies.to_csv('extracted_popularity_score.csv', index=False)

In [30]:
invalid_movies

,movie_name
0,Open Season
1,GoldenEye
2,Die Another Day
3,Tomorrow Never Dies
4,The World Is Not Enough
...,...
6800,The Sweet Hereafter
6801,Ride with the Devil
6802,Encino Man
6803,Monkeybone


In [11]:
len(df)

13774

In [3]:
import pandas as pd
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')
# Ensure necessary NLTK data files are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')


# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet
# Extended stopwords
additional_stopwords = {
    'movie', 'film', 'movies', 'films', 'however', 'character', 'characters',
    'story', 'stories', 'plot', 'scene', 'scenes', 'year', 'years','begin',
    'becomes','back','also','boy','girl','new','along','around','belong','another',
    'away','best','become','come','across'
}

def preprocess_text(description):
    if pd.isnull(description):
        return []

    # Tokenize the description
    words = word_tokenize(description)

    # Convert to lower case
    words = [word.lower() for word in words]

    # Remove punctuation
    words = [word for word in words if word.isalpha()]

    # Remove stopwords
    stop_words = set(stopwords.words('english')).union(additional_stopwords)
    words = [word for word in words if word not in stop_words]

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return words

def filter_proper_nouns(words):
    # POS tag the words
    pos_tags = pos_tag(words)

    # Filter out proper nouns (NNP or NNPS)
    filtered_words = [word for word, pos in pos_tags if pos not in ('NNP', 'NNPS')]
    
    return ' '.join(filtered_words)

def extract_tags(description, num_tags=10):
    if pd.isnull(description):
        return []

    # Preprocess the text
    preprocessed_words = preprocess_text(description)

    if not preprocessed_words:
        return []

    # Filter out proper nouns
    filtered_description = filter_proper_nouns(preprocessed_words)

    if not filtered_description.strip():
        # If filtered description is empty, fallback to original preprocessed text
        filtered_description = ' '.join(preprocessed_words)

    # Initialize the TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()
    try:
        tfidf_matrix = vectorizer.fit_transform([filtered_description])
    except ValueError:
        return []

    # Get feature names (words)
    feature_names = vectorizer.get_feature_names_out()

    # Get the scores for each word
    scores = tfidf_matrix.toarray()[0]

    # Create a dictionary of words and their scores
    word_score_dict = dict(zip(feature_names, scores))

    # Sort the words by their scores in descending order
    sorted_words = sorted(word_score_dict.items(), key=lambda item: item[1], reverse=True)

    # Extract up to num_tags words
    tags = [word for word, score in sorted_words[:num_tags]]

    return tags


# Apply the function to the DataFrame and create a new column 'tags'
df['tags'] = df['description'].apply(lambda x: extract_tags(x, num_tags=10))



[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /kaggle/working/corpora/wordnet.zip
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...


replace /kaggle/working/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [16]:
len(df)

7858

In [17]:
df

,movie_name,movie_id,runtime,genre,rating,director,star,gross(in $),description,audience_score,critics_score,where_to_watch,popularity_score,release_date,production_companies,budget($),more_like_this,release_year,invalid_more_like_this,tags
0,The Abandoned,tt0475937,99 min,"[Horror, Mystery, Thriller]",5.5,Nacho Cerdà,"[Anastasia Hille, Karel Roden, Valentin Goshev...",1255524.0,"Marie (Anastasia Hille), a film producer, retu...",4.2,3.6,[],NaN,"February 23, 2007","[Castelao Producciones, Filmax International, ...",NaN,"[The Tripper, The Abandoned]",2007.0,"[The Gravedancers, Dark Ride, Wicked Little Th...","[anastasia, birth, brother, claim, dead, doppe..."
1,Flannel Pajamas,tt0432290,124 min,[Romance],6.0,Jeff Lipsky,"[Justin Kirk, Julianne Nicholson, Chelsea Altm...",123301.0,A dinner date evolves into love for a New York...,3.9,5.1,[],NaN,"January 22, 2006","[Gigantic Pictures, Plainview Pictures]",NaN,"[Sophie and the Rising Sun, Tully, Seeing Othe...",2006.0,"[Her Name Is Carla, Seagull, Shadows & Lies, I...","[couple, begin, business, catering, date, dete..."
2,The Hoax,tt0462338,116 min,"[Biography, Comedy, Drama]",6.7,Lasse Hallström,"[Richard Gere, Alfred Molina, David Aaron Bake...",7156933.0,"In the early 1970s, author Clifford Irving (Ri...",5.8,8.6,"[{'platform': 'amazon-prime-video-us', 'href':...",NaN,"April 20, 2007","[Miramax, Bob Yari Productions, The Mark Gordo...",25000000.0,"[Miles from Home, Intersection, Norman, Arbitr...",2007.0,"[The Hunting Party, No Mercy, Red Corner, The ...","[hughes, richard, accomplice, alfred, attempt,..."
3,Samoan Wedding,tt0464184,97 min,"[Comedy, Romance]",6.3,Chris Graham,"[Oscar Kightley, Iaheto Ah Hi, Shimpal Lelisi,...",71840.0,Four miscreant friends are banned from a weddi...,8.0,8.3,"[{'platform': 'amazon-prime-video-us', 'href':...",NaN,"March 30, 2006",[South Pacific Pictures],2000000.0,[Sione's 2: Unfinished Business],2006.0,"[Once Were Warriors, Bro'Town, The Breaker Upp...","[banned, four, friend, girlfriend, miscreant, ..."
4,Firewall,tt0408345,105 min,"[Action, Crime, Thriller]",5.8,Richard Loncraine,"[Harrison Ford, Virginia Madsen, Paul Bettany,...",48751189.0,Bank security expert Jack Stanfield (Harrison ...,3.7,1.9,"[{'platform': 'amazon-prime-video-us', 'href':...",NaN,"February 10, 2006","[Warner Bros., Village Roadshow Pictures, Beac...",50000000.0,"[Hollywood Homicide, Crossing Over, Presumed I...",2006.0,"[Extraordinary Measures, Frantic, K-19: The Wi...","[system, balance, bank, bettany, break, build,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8241,The 24th Day,tt0246404,92.0,"[Crime, Drama, Thriller]",6.4,Tony Piccirillo,"[James Marsden, Scott Speedman, Sofía Vergara,...",14460.0,A widower (Scott Speedman) with HIV holds a fo...,5.8,2.7,[],NaN,"May 6, 2004","[Big Teddy Films, Nazz Productions]",1000000.0,"[Heights, Adoration, Citizen Gangster, 10th & ...",2004.0,"[The Broken Hearts Club: A Romantic Comedy, We...","[aid, force, former, hiv, hold, hostage, james..."
8242,The Ring Two,tt0377109,110.0,"[Horror, Mystery]",5.4,Hideo Nakata,"[Naomi Watts, David Dorfman, Sissy Spacek, Sim...",76231249.0,A female journalist (Naomi Watts) must prevent...,3.3,2.1,"[{'platform': 'amazon-prime-video-us', 'href':...",NaN,"March 18, 2005","[Dreamworks Pictures, BenderSpink, Parkes/MacD...",50000000.0,"[Rings, Rings, The Grudge, The Grudge 2, Annab...",2005.0,"[The Ring, Ring 2, Ringu, The Grudge 3]","[david, dorfman, evil, female, journalist, mus..."
8243,Rings,tt0498381,102.0,"[Drama, Horror, Mystery]",4.5,F. Javier Gutiérrez,"[Matilda Anna Ingrid Lutz, Alex Roe, Johnny Ga...",27793018.0,A young woman (Matilda Lutz) becomes worried a...,2.3,0.8,"[{'platform': 'amazon-prime-video-us', 'href':...",NaN,"February 3, 2017","[Macari/Edelstein, Parkes/MacDonald Image Nati...",25000000.0,"[The Ring Two, Rings, The Grudge 2, The Grudge...",2017.0,"[The Ring, The Grudge 3, Ring 2, Ringu, Ju-on:...","[boyfriend, alex, dark, day, di

In [4]:
df.drop(columns=["description","release_date","runtime","budget($)","gross(in $)","where_to_watch","gross(in $)","critics_score"], inplace=True)

In [5]:
df.drop(columns=["invalid_more_like_this"], inplace=True)

In [6]:
from collections import Counter

all_tags = [tag for sublist in df['tags'] for tag in sublist]
tag_counts = Counter(all_tags)

# Get the top 50 tags
top_50_tags = tag_counts.most_common(50)

# Convert to DataFrame for better readability
top_50_tags_df = pd.DataFrame(top_50_tags, columns=['Tag', 'Frequency'])

print(top_50_tags_df)

          Tag  Frequency
0        find       1222
1      friend        874
2      family        820
3        life        786
4         get        665
5      father        565
6        help        484
7       begin        483
8          go        467
9        home        432
10       city        426
11   daughter        423
12        man        417
13       fall        416
14        day        412
15      child        389
16      death        388
17    brother        384
18     former        341
19      force        325
20  discovers        322
21      first        316
22       love        309
23   american        303
24      group        301
25       john        300
26      agent        292
27    decides        290
28      woman        282
29        one        274
30       even        269
31        end        254
32      crime        254
33     battle        252
34    angeles        247
35      david        245
36    attempt        243
37      fight        240
38       must        239


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer



# Define weights for the features
feature_weights = {
    'popularity_score': 2,
    'release_year': 1.5,
    'genre': 5,
    'director': 3,
    'star': 2,
    'audience_score':0.5,
    'production_companies': 2,
    'tags':2
}

# Define numerical and categorical features
numerical_features = ['popularity_score','release_year','audience_score']
categorical_features = ['genre', 'director', 'star', 'production_companies','tags']

# Preprocess numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
director_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer())
])
# Create the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('genre', MultiLabelBinarizerWrapper(), 'genre'),
        ('director', director_transformer, 'director'),
        ('star', MultiLabelBinarizerWrapper(), 'star'),
        ('production_companies', MultiLabelBinarizerWrapper(), 'production_companies'),
        ('tags', MultiLabelBinarizerWrapper(), 'tags')


    ]
)

# Fit the preprocessor to get the transformed column shapes
preprocessed = preprocessor.fit_transform(df)

# Compute weights dynamically
def compute_weights(preprocessor, df, feature_weights):
    preprocessed = preprocessor.transform(df)
    weights = []
    current_col = 0
    for name, transformer, columns in preprocessor.transformers:
        if name == 'num':
            for col in columns:
                weights.extend([feature_weights[col]] * preprocessed[:, current_col:current_col + 1].shape[1])
                current_col += 1
        else:
            col_weight = feature_weights[columns]
            mlb_transformed_shape = transformer.fit_transform(df[columns]).shape[1]
            weights.extend([col_weight] * mlb_transformed_shape)
            current_col += mlb_transformed_shape
    return np.array(weights)

# Compute the weights
weights = compute_weights(preprocessor, df, feature_weights)

# Apply weights function
def apply_weights(X, weights):
    return X * weights

# Apply preprocessing and weighting in a single pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('weigher', FunctionTransformer(apply_weights, kw_args={'weights': weights}, validate=False))
])

# Apply the pipeline to the dataset
processed_features = pipeline.fit_transform(df)

# Display the processed features
print(processed_features.shape)
print(processed_features[:5])

(7858, 40440)
[[-1.15424452e-15  5.91997121e-01 -4.31818061e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.15424452e-15  5.01880437e-01 -5.05847711e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.15424452e-15  5.91997121e-01 -3.69932599e-02 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.15424452e-15  5.01880437e-01  5.05890842e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.15424452e-15  5.01880437e-01 -5.55200811e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


# **Aggregating More_like_this list**

In [6]:
len(processed_features[5])

46255

In [6]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
def get_aggregate_features(movie_list, df, pipeline):
    similar_movies = df[df['movie_name'].isin(movie_list)]

    if similar_movies.empty:
        # If no similar movies are found, return an array of zeros with the expected shape
        n_features = processed_features.shape[1]  # Use the shape of processed features
        return np.zeros(n_features)

    # Get the features of similar movies without additional preprocessing
    similar_movie_features = processed_features[df.index.isin(similar_movies.index)]

    # Return the mean of the features
    return np.mean(similar_movie_features, axis=0)

# Apply the function to create aggregated features for each movie
aggregated_features = []
for _, row in df.iterrows():
    similar_movies = row['more_like_this']
    features = get_aggregate_features(similar_movies, df, pipeline)
    aggregated_features.append(features)

# Add the aggregated features to the DataFrame

# Convert the list of feature arrays to a 2D numpy array
target = np.array(aggregated_features)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(processed_features, target, test_size=0.1, random_state=42)
del target
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (7072, 40440)
X_test shape: (786, 40440)
y_train shape: (7072, 40440)
y_test shape: (786, 40440)


In [8]:
X_train

array([[-1.15424452e-15, -2.29173679e+00, -3.69932599e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.15424452e-15,  7.72230491e-01, -5.30524261e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-4.96500028e+00, -8.49869833e-01,  8.76039093e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-1.18814300e+00, -8.49869833e-01, -1.85052560e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.15424452e-15,  1.76351402e+00,  9.74745293e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.15424452e-15,  1.31293060e+00, -8.63463600e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam



# Ensure your data is correctly loaded and preprocessed
# Assuming X_train, y_train, X_test, y_test are already loaded

# Define the model within the TPU strategy scope
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(X_test.shape[1], activation='linear')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mse'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.2, verbose=1)

# Evaluate the model
loss, mse = model.evaluate(X_test, y_test, verbose=1)
print(f'Model Mean Squared Error: {mse}')

# Make predictions on the test set
predictions = model.predict(X_test)


2024-07-14 23:08:57.579290: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 23:08:57.579431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 23:08:57.847243: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Epoch 1/50
 7/45 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0017 - mse: 0.0017

I0000 00:00:1720998561.764177    1351 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720998561.783235    1351 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.0017 - mse: 0.0017

W0000 00:00:1720998567.956545    1352 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


45/45 ━━━━━━━━━━━━━━━━━━━━ 12s 153ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.0016 - val_mse: 0.0016
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0013 - val_mse: 0.0013
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0014 - mse: 0.0014 - val_loss: 0.0013 - val_mse: 0.0013
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.0012 - val_mse: 0.0012
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0011 - val_mse: 0.0011
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.0010 - val_mse: 0.0010
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 9.8893e-04 - mse: 9.8893e-04 - val_loss: 0.0010 - val_mse: 0.0010
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 9.2686e-04 - mse: 9.2686e-04 - val_loss: 9.8553e-04 - val_mse: 9.8553e-04
Epoch 9/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/ste

In [44]:
df.columns

Index(['movie_name', 'movie_id', 'genre', 'rating', 'director', 'star',
       'audience_score', 'popularity_score', 'production_companies',
       'more_like_this', 'release_year', 'tags'],
      dtype='object')

In [10]:
import re
import requests
from bs4 import BeautifulSoup

def extract_movie_id_from_imdb_link(imdb_link):
    """
    Extract the movie ID from IMDb link.
    
    :param imdb_link: URL of the IMDb movie page
    :return: Movie ID as string
    """
    match = re.search(r'tt\d+', imdb_link)
    if match:
        return match.group(0)
    else:
        raise ValueError("Invalid IMDb link. Movie ID not found.")

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

def get_movie_details(movie_id):
    """
    Fetch movie details using IMDb ID.
    
    :param movie_id: IMDb movie ID
    :return: DataFrame containing movie details
    """
    # Implement the function to fetch movie details
    pass

def recommend_movies_interactive(df, model, preprocessor, top_n=5):
    """
    Recommend movies based on user input (IMDb link or movie name).
    
    :param df: DataFrame containing movie data
    :param model: Trained model for generating features
    :param preprocessor: Preprocessor object used for feature scaling
    :param top_n: Number of top similar movies to recommend
    :return: List of tuples with recommended movie names and their similarity percentages
    """
    movie_ids_or_names = []

    while True:
        
        imdb_link = input("Please enter the IMDb link: ").strip()
        try:
            movie_id = extract_movie_id_from_imdb_link(imdb_link)
            print(f"Extracted movie ID: {movie_id}")
            movie_ids_or_names.append(movie_id)
        except ValueError as e:
            print(e)

        
        add_more = input("Do you want to add another movie? (yes/no): ").strip().lower()
        if add_more == 'no':
            break

    recommendations = {}

    for movie_id_or_name in movie_ids_or_names:
        # Check if the movie exists in the dataset
        test_movies = df[df['movie_id'] == movie_id_or_name] if movie_id_or_name.startswith('tt') else df[df['movie_name'] == movie_id_or_name]
        
        if test_movies.empty:
            test_movies = get_movie_details(movie_id_or_name)

        if test_movies.empty:
            print(f"Movie {movie_id_or_name} not found in dataset or via IMDb.")
            continue

        # Print the names of the found movies
        found_movie_names = test_movies['movie_name'].tolist()
        print(f"Found movies in the dataset: {', '.join(found_movie_names)}")

        # Preprocess the features of the selected movies
        test_movie_features = preprocessor.transform(test_movies.drop(columns=['movie_name', 'movie_id']))

        # Predict the features for the "more like this" movies
        predicted_features = model.predict(test_movie_features)

        # Calculate cosine similarity between the predicted features and all movies in the dataset
        all_movie_features = preprocessor.transform(df.drop(columns=['movie_name', 'movie_id']))
        similarities = cosine_similarity(predicted_features, all_movie_features).flatten()

        # Normalize similarities to get percentage
        similarity_percentages = (similarities / similarities.max()) * 100

        # Get the indices of the top similar movies, excluding the input movies
        similar_movie_indices = [idx for idx in similarities.argsort()[::-1] if df.iloc[idx]['movie_id'] != movie_id_or_name and df.iloc[idx]['movie_name'] != movie_id_or_name][:top_n]

        # Store recommendations
        for idx in similar_movie_indices:
            movie_name = df.iloc[idx]['movie_name']
            similarity = similarity_percentages[idx]
            if movie_name in recommendations:
                recommendations[movie_name].append(similarity)
            else:
                recommendations[movie_name] = [similarity]

    # Prioritize movies that appear in recommendations for multiple input movies
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-len(x[1]), -np.mean(x[1])))

    # Format the final recommendations
    final_recommendations = [(movie, np.mean(similarities)) for movie, similarities in sorted_recommendations][:top_n]

    return final_recommendations

# Example usage:
# recommended_movies = recommend_movies_interactive(df, model, preprocessor, top_n=5)
# print(recommended_movies)


In [11]:
recommended_movies = recommend_movies_interactive(df, model, preprocessor, top_n=20)
print(recommended_movies)

Please enter the IMDb link:  https://www.imdb.com/title/tt3104988/


Extracted movie ID: tt3104988


Do you want to add another movie? (yes/no):  no


Found movies in the dataset: Crazy Rich Asians
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
[('Crazy, Stupid, Love.', 100.0), ('Love & Other Drugs', 95.48958720755687), ('Poor Things', 94.93949681561934), ('How to Be Single', 94.93581117205612), ('She Came to Me', 92.59345542167937), ('500 Days of Summer', 92.45684266615613), ('Silver Linings Playbook', 91.44354133865201), ('50 First Dates', 91.17218210237856), ('Asteroid City', 91.07316182990832), ('The Longest Ride', 90.37763094693177), ('Cashback', 90.04240642156338), ('Vicky Cristina Barcelona', 88.37960333843867), ('Pretty in Pink', 88.16853791571124), ('Don Jon', 87.22934180563492), ('About Time', 87.17878268013351), ('The Proposal', 87.0153762039139), ('New York, I Love You', 86.66435303539215), ('Bridesmaids', 86.51183764130768), ('Secretary', 86.06821477055063), ('This Is 40', 85.85527884201998)]


In [12]:
import joblib
from tensorflow.keras.models import load_model

# Assuming you have defined your model and pipeline
model.save('Item_item_collaborative_filtering_model.h5')
joblib.dump(pipeline, 'Item_item_collaborative_preprocessor.pkl')
# Save the DataFrame
df.to_pickle('Item_item_collaborative_data.pkl')

In [11]:
import re
import joblib
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.
import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import load_model

# Define preprocessing functions and classes
def clean_percentage(x):
    if isinstance(x, str):
        return float(x.strip('%')) / 100
    return x

def clean_comma(x):
    if isinstance(x, str):
        return float(x.replace(',', ''))
    return x

class MultiLabelBinarizerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        self.mlb.fit(X)
        return self

    def transform(self, X):
        return self.mlb.transform(X)

# Load the dataset

"""model = joblib.load('/kaggle/input/model-weights/scikitlearn/model-weights-genre/1/model.pkl')

# Load the preprocessor
preprocessor = joblib.load('/kaggle/input/preprocessor-genre/preprocessor.pkl')"""


# Example usage:
recommended_movies = recommend_movies_interactive(df, model, preprocessor, top_n=50)
print(recommended_movies)


Enter '1' if you want to provide an IMDb link or '2' for a movie name:  1
Please enter the IMDb link:  https://www.imdb.com/title/tt3104988/


Extracted movie ID: tt3104988


Do you want to add another movie? (yes/no):  no


Found movies in the dataset: Crazy Rich Asians
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
[('How to Be Single', 100.0), ('Silver Linings Playbook', 98.02184175446249), ('The Proposal', 97.89082206269161), ('50 First Dates', 97.07087836212138), ('500 Days of Summer', 96.82700677061169), ('Poor Things', 96.31365787325828), ('She Came to Me', 95.7268986274751), ('New York, I Love You', 95.20565564131562), ('The Longest Ride', 94.9461359310141), ('Asteroid City', 94.69918310908628), ('Vicky Cristina Barcelona', 93.40529149894635), ('Don Jon', 93.27715917099619), ('This Is 40', 93.11406991542125), ('Pretty in Pink', 93.10204546877428), ('Forgetting Sarah Marshall', 92.05701028258602), ('Easy A', 91.6417219914122), ('Sex and the City', 91.01355538120194), ('Sideways', 90.70109632676687), ("He's Just Not That Into You", 90.65431288656018), ('Secretary', 90.05686477024236), ("You've Got Mail", 87.97192722635307), ('Jerry Maguire', 87.96493630318145), ('Bridesmaids', 87.59337241788421), ('Mamma Mia

In [14]:
model.save('movie_recommendation_model-latest.h5')


In [ ]:
import random
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd

# List of user agents
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:91.0) Gecko/20100101 Firefox/91.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:92.0) Gecko/20100101 Firefox/92.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36',

]

# Function to get random user agent
def get_random_user_agent():
    return random.choice(user_agents)

# Function to format movie name
def format_movie_name(movie_name):
    formatted_name = re.sub(r'[^a-zA-Z0-9\s]', '', movie_name).replace(' ', '_').lower()
    return formatted_name

# Function to extract year from date string
def extract_year(date_str):
    if pd.isnull(date_str) or date_str == '':
        return ''
    try:
        date_obj = datetime.strptime(date_str, '%B %d, %Y')
        return str(date_obj.year)
    except ValueError:
        return ''

def convert_to_minutes(runtime_text):
    parts = runtime_text.split()
    hours = 0
    minutes = 0
    for i in range(0, len(parts), 2):
        if 'hour' in parts[i+1]:
            hours = int(parts[i])
        elif 'minute' in parts[i+1]:
            minutes = int(parts[i])
    return hours * 60 + minutes
def str_to_list(s):
    s = s.strip()  # Remove leading/trailing whitespace
    if s.startswith('[') and s.endswith(']'):
        try:
            return ast.literal_eval(s)
        except (ValueError, SyntaxError):
            return s  # Return the original string if evaluation fails
    return s  # Return the original string if it's not a list format

# Function to fetch movie page
def fetch_movie_page(url, headers):
    response = requests.get(url, headers=headers)
    return response

# Function to scrape movie details from IMDb
def scrape_movie_details(session, movie_id):
    base_url = f"https://www.imdb.com/title/{movie_id}/"
    headers = {
        'User-Agent': get_random_user_agent(),
        'Accept-Language': 'en-US,en;q=0.9',
    }
    try:
        response = session.get(base_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract required fields
        img_tag = soup.select_one('img.ipc-image[loading="eager"]')
        poster_url = img_tag['src'] if img_tag else None

        popularity_tag = soup.select_one('div[data-testid="hero-rating-bar__popularity__score"]')
        popularity_score = popularity_tag.text.strip() if popularity_tag else None

        release_date_tag = soup.select_one('a.ipc-metadata-list-item__list-content-item[href*="/releaseinfo?ref_=tt_dt_rdat"]')
        release_date = release_date_tag.text.strip().split('(')[0].strip() if release_date_tag else None

        production_companies_tags = soup.select('a.ipc-metadata-list-item__list-content-item[href*="?ref_=tt_dt_co_"]')
        production_companies = [tag.text.strip() for tag in production_companies_tags]

        genres = [genre.text for genre in soup.select('div.ipc-chip-list__scroller > a')]
        director_tag = soup.find('a', {'href': lambda x: x and 'tt_cl_dr_1' in x})
        director = director_tag.text.strip() if director_tag else 'N/A'
        top_cast_tags = soup.select('a[data-testid="title-cast-item__actor"]')[:6]
        top_cast = [tag.text.strip() for tag in top_cast_tags]

        # Extract gross revenue
        gross_revenue_tag = soup.find('li', {'data-testid': 'title-boxoffice-grossdomestic'})
        if gross_revenue_tag:
            gross_revenue_text = gross_revenue_tag.find_all('span')[1].text
            gross_revenue = int(gross_revenue_text.replace('$', '').replace(',', ''))
        else:
            gross_revenue = 'N/A'

        # Extract rating
        rating_tag = soup.find('span', {'class': 'sc-bde20123-1 cMEQkK'})
        rating = float(rating_tag.text) if rating_tag else 'N/A'

        budget_tag = soup.select_one('li[data-testid="title-boxoffice-budget"] span.ipc-metadata-list-item__list-content-item')
        budget = budget_tag.text.strip() if budget_tag else None
        if budget:
            budget = budget.replace('$', '').replace('(estimated)', '').replace(',', '').strip()
            try:
                budget = int(budget)
            except ValueError:
                budget = None

        movie_name_tag = soup.select_one('h1[data-testid="hero__pageTitle"] span')
        movie_name = movie_name_tag.text.strip() if movie_name_tag else None

        runtime_tag = soup.find('li', {'data-testid': 'title-techspec_runtime'})
        if runtime_tag:
            runtime_text = runtime_tag.find_all('div')[0].text
            runtime_minutes = convert_to_minutes(runtime_text)
        else:
            runtime_minutes = 'N/A'

        # Fetch 'more like this' movies
        more_like_this_movies = []
        movies = soup.find_all('a', class_='ipc-poster-card__title ipc-poster-card__title--clamp-2 ipc-poster-card__title--clickable')
        for movie in movies:
            if movie.find('span'):
                more_like_this_movies.append(movie.find('span').text.strip())

        if not poster_url or not production_companies:
            return None

    except requests.RequestException as e:
        return None

    return {
        "movie_id": movie_id,
        'movie_name': movie_name,
        "star": top_cast,
        'poster_url': poster_url,
        "genre": genres,
        "rating": rating,
        "director": director,
        "gross(in $)": gross_revenue,
        "runtime": runtime_minutes,
        "more_like_this": more_like_this_movies,
        "release_date": release_date,
        'popularity_score': popularity_score,
        'production_companies': production_companies,
        'budget': budget
    }

# Function to get movie details from Rotten Tomatoes
def get_movie_details_rt(movie_name, headers, year=""):
    formatted_name = format_movie_name(movie_name)

    search_url = f"https://www.rottentomatoes.com/search?search={movie_name.replace(' ', '%20')}%20{year}"
    search_response = requests.get(search_url, headers=headers)
    search_soup = BeautifulSoup(search_response.content, 'html.parser')
    search_results = search_soup.find('ul', {'slot': 'list'})
    if search_results:
        first_result = search_results.find('search-page-media-row')
        if first_result:
            rt_link = first_result.find('a', {'data-qa': 'thumbnail-link'})['href']
            if not rt_link.startswith('https://www.rottentomatoes.com'):
                formatted_name = rt_link
            response = fetch_movie_page(rt_link, headers)
        else:
            return None
    else:
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    synopsis_section = soup.find('drawer-more', {'slot': 'description'})
    synopsis = synopsis_section.text.strip().replace('Read More', '').replace('Read Less', '').strip() if synopsis_section else "Synopsis not found."

    audience_score_section = soup.find('rt-button', {'slot': 'audienceScore'})
    audience_score = audience_score_section.find('rt-text').text.strip() if audience_score_section else "Audience score not found."

    critics_score_section = soup.find('rt-button', {'slot': 'criticsScore'})
    critics_score = critics_score_section.find('rt-text').text.strip() if critics_score_section else "Critics score not found."

    where_to_watch_section = soup.find('section', {'data-qa': 'section:where-to-watch'})
    where_to_watch = []
    if where_to_watch_section:
        affiliates = where_to_watch_section.find_all('where-to-watch-meta', {'data-qa': 'affiliate-item'})
        for affiliate in affiliates:
            platform = affiliate['affiliate']
            if any(word in platform.lower() for word in ['prime', 'netflix', 'disney', 'max']):
                href = affiliate['href']
                where_to_watch.append({'platform': platform, 'href': href})

    original_language = None
    category_wraps = soup.find_all('div', class_='category-wrap')
    for category in category_wraps:
        key = category.find('rt-text', class_='key')
        if key:
            key_text = key.text.strip()
            value = category.find('dd').find('rt-text').text.strip() if category.find('dd').find('rt-text') else ""
            if key_text == "Original Language":
                original_language = value

    return {
        'movie': movie_name,
        'original_language': original_language,
        'description': synopsis,
        'audience_score': audience_score,
        'critics_score': critics_score,
        'where_to_watch': where_to_watch
    }

# Main function to get all movie details
def get_movie_details(imdb_id):
    headers = {
        'User-Agent': get_random_user_agent(),
        'Accept-Language': 'en-US,en;q=0.9',
    }

    with requests.Session() as session:
        imdb = scrape_movie_details(session, imdb_id)

    if not imdb:
        return None

    rt = get_movie_details_rt(imdb['movie_name'], headers, extract_year(imdb['release_date']))

    combined_data = []
    combined_entry = {
        'movie_id': imdb['movie_id'],
        'movie': imdb['movie_name'],
        'movie_name': imdb['movie_name'],
        'poster_url': imdb['poster_url'],
        'star': imdb['star'],
        'genre': imdb['genre'],
        'rating': imdb['rating'],
        'director': imdb['director'],
        'gross(in $)': imdb['gross(in $)'],
        'runtime': imdb['runtime'],
        'audience_score': rt.get('audience_score', ''),
        'critics_score': rt.get('critics_score', ''),
        'description': rt.get('description', ''),
        'where_to_watch': ', '.join([f"{wtw['platform']} ({wtw['href']})" for wtw in rt.get('where_to_watch', [])]),
        'more_like_this': imdb['more_like_this'],
        'release_date': imdb['release_date'],
        'release_year': extract_year(imdb['release_date']),
        'popularity_score': imdb['popularity_score'],
        'production_companies': imdb['production_companies'],
        'budget($)': imdb['budget']
    }
    combined_data.append(combined_entry)

    desired_order = [
        'movie_id', 'movie_name', 'runtime', 'genre', 'rating', 'director', 'star', 
        'gross(in $)', 'movie', 'description', 'audience_score', 'critics_score', 
        'where_to_watch', 'poster_url', 'popularity_score', 'release_date', 
        'production_companies', 'budget($)', 'more_like_this', 'release_year'
    ]

    # Create DataFrame with specified order
    df = pd.DataFrame(combined_data)[desired_order]
    
    # Apply the function to the column

    # Fill missing values and clean data
    df['critics_score'] = df['critics_score'].apply(lambda x: np.nan if isinstance(x, str) and 'not found' in x else x)
    df['audience_score'] = df['audience_score'].apply(lambda x: np.nan if isinstance(x, str) and 'not found' in x else x)
    df['audience_score'] = df['audience_score'].apply(clean_percentage)
    df['critics_score'] = df['critics_score'].apply(clean_percentage)
    df['audience_score'] = df['audience_score'].fillna(df['audience_score'].mean())
    df['critics_score'] = df['critics_score'].fillna(df['critics_score'].mean())
    df['description'] = df['description'].fillna('')
    df['popularity_score'] = df['popularity_score'].apply(clean_comma)
    df['gross(in $)'] = df['gross(in $)'].apply(clean_comma)
    df['star'] = df['star'].apply(lambda x: x.split(', \n') if isinstance(x, str) else x if isinstance(x, list) else [])
    df['genre'] = df['genre'].apply(lambda x: x if isinstance(x, list) else x.split(', ') if isinstance(x, str) else [])
    df['release_year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year
    df['release_year'] = df['release_year'].fillna(df['release_year'].median())
    df['tags'] = df['description'].apply(lambda x: extract_tags(x, num_tags=5))
    return df



In [22]:
import pandas as pd
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize

# Ensure necessary NLTK data files are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Extended stopwords
additional_stopwords = {
    'movie', 'film', 'movies', 'films', 'however', 'character', 'characters',
    'story', 'stories', 'plot', 'scene', 'scenes', 'year', 'years','begin',
    'becomes','back','also','boy','girl','new','along','around','belong','another',
    'away','best','become','come','across'
}

def preprocess_text(description):
    if pd.isnull(description):
        return []

    # Tokenize the description
    words = word_tokenize(description)

    # Convert to lower case
    words = [word.lower() for word in words]

    # Remove punctuation
    words = [word for word in words if word.isalpha()]

    # Remove stopwords
    stop_words = set(stopwords.words('english')).union(additional_stopwords)
    words = [word for word in words if word not in stop_words]

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return words

def filter_proper_nouns(words):
    # POS tag the words
    pos_tags = pos_tag(words)

    # Filter out proper nouns (NNP or NNPS)
    filtered_words = [word for word, pos in pos_tags if pos not in ('NNP', 'NNPS')]
    
    return ' '.join(filtered_words)

def extract_tags(description, num_tags=10):
    if pd.isnull(description):
        return []

    # Preprocess the text
    preprocessed_words = preprocess_text(description)

    if not preprocessed_words:
        return []

    # Filter out proper nouns
    filtered_description = filter_proper_nouns(preprocessed_words)

    if not filtered_description.strip():
        # If filtered description is empty, fallback to original preprocessed text
        filtered_description = ' '.join(preprocessed_words)

    # Initialize the TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()
    try:
        tfidf_matrix = vectorizer.fit_transform([filtered_description])
    except ValueError:
        return []

    # Get feature names (words)
    feature_names = vectorizer.get_feature_names_out()

    # Get the scores for each word
    scores = tfidf_matrix.toarray()[0]

    # Create a dictionary of words and their scores
    word_score_dict = dict(zip(feature_names, scores))

    # Sort the words by their scores in descending order
    sorted_words = sorted(word_score_dict.items(), key=lambda item: item[1], reverse=True)

    # Extract up to num_tags words
    tags = [word for word, score in sorted_words[:num_tags]]

    return tags

# Sample movie data
movies = {
    'movie_id': ['tt0111161', 'tt0068646', 'tt0071562', 'tt0468569', 'tt0050083'],
    'movie_name': ['The Shawshank Redemption', 'The Godfather', 'The Godfather: Part II', 'The Dark Knight', '12 Angry Men'],
    'description': [
        'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
        'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.',
        'The early life and career of Vito Corleone in 1920s New York City is portrayed while his son, Michael, expands and tightens his grip on the family crime syndicate.',
        'When the menace known as the Joker emerges from his mysterious past, he wreaks havoc and chaos on the people of Gotham.',
        'A jury holdout attempts to prevent a miscarriage of justice by forcing his colleagues to reconsider the evidence.'
    ]
}

df = pd.DataFrame(movies)

# Apply the function to the DataFrame and create a new column 'tags'
df['tags'] = df['description'].apply(lambda x: extract_tags(x, num_tags=10))

# Print the DataFrame to see the extracted tags
print(df[['movie_name', 'tags']])


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
                 movie_name                                               tags
0  The Shawshank Redemption  [act, bond, common, decency, eventual, finding...
1             The Godfather  [aging, clandestine, control, crime, dynasty, ...
2    The Godfather: Part II  [career, city, corleone, crime, early, expands...
3           The Dark Knight  [chaos, emerges, gotham, havoc, joker, known, ...
4              12 Angry Men  [attempt, colleague, ev